In [1]:
import langchain
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS, pinecone
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from tqdm.autonotebook import tqdm

load_dotenv()

/home/kelechi/code/machine_learning/pharmassist_v0(rag)/.pharmassist_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-flash", google_api_key=os.getenv("GOOGLE_API_KEY")
)

llm_gemini

ChatGoogleGenerativeAI(model='models/gemini-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x729bcc246290>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x729bcc298df0>, default_metadata=())

In [8]:
pdf_pages = []
docs_path = "pdfs/PMB271"
for file in tqdm(os.listdir(docs_path)):
    file_path = os.path.join(docs_path, file)
    pdf_loader = PyPDFLoader(file_path, extract_images=True)
    pdf_pages += pdf_loader.load_and_split()

faiss_index = FAISS.from_documents(pdf_pages, GoogleGenerativeAIEmbeddings())

In [ ]:
print(len(pdf_pages))